<a href="https://colab.research.google.com/github/BinarySstar/Credit-Card-Segment-Classfication/blob/master/Credit_Card_Segment_Classfication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive Mount
train 및 test 데이터를 drive에 저장하기 위해 mount를 진행

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# EDA
## 데이터 로드
- parquet 형식의 데이터를 로드

In [2]:
!pip install fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.8 MB/s eta 0:00:00


- 전체 parquet 파일에 대한 요약

In [6]:
import pandas as pd
import os
from glob import glob

# Google Drive 경로 설정
base_path = "drive/MyDrive/고급프로그래밍/train"

# 폴더 이름 정의
folder_name = ["회원정보", "신용정보", "승인매출정보", "청구입금정보", "잔액정보", "채널정보", "마케팅정보", "성과정보"]
folders = [f"{i}.{name}" for i, name in enumerate(folder_name, start=1)]

# 요약 결과를 정리하는 list
summary_list = []

for folder in folders:
    folder_path = os.path.join(base_path, folder)
    parquet_files = glob(os.path.join(folder_path, "*.parquet"))

    for file in parquet_files:
        try:
            df = pd.read_parquet(file, engine="fastparquet")

            info = {
                "폴더명": folder,
                "파일명": file,
                "행 수": df.shape[0],
                "열 수": df.shape[1],
                "컬럼 목록": df.columns.tolist(),
                "데이터 타입": df.dtypes.to_dict(),
                "결측치 비율": df.isnull().mean().to_dict(),
                "샘플 데이터": df.head().to_dict()
            }

            summary_list.append(info)
            print(f"✅ {file} 로드 완료")

        except Exception as e:
            print(f"❌ {file} 처리 중 오류: {e}")

# 결과 DataFrame으로 정리
summary_df = pd.DataFrame(summary_list)
summary_df

✅ drive/MyDrive/고급프로그래밍/train/1.회원정보/201809_train_회원정보.parquet 로드 완료
✅ drive/MyDrive/고급프로그래밍/train/1.회원정보/201808_train_회원정보.parquet 로드 완료
✅ drive/MyDrive/고급프로그래밍/train/1.회원정보/201807_train_회원정보.parquet 로드 완료
✅ drive/MyDrive/고급프로그래밍/train/1.회원정보/201810_train_회원정보.parquet 로드 완료
✅ drive/MyDrive/고급프로그래밍/train/1.회원정보/201812_train_회원정보.parquet 로드 완료
✅ drive/MyDrive/고급프로그래밍/train/1.회원정보/201811_train_회원정보.parquet 로드 완료
✅ drive/MyDrive/고급프로그래밍/train/2.신용정보/201807_train_신용정보.parquet 로드 완료
✅ drive/MyDrive/고급프로그래밍/train/2.신용정보/201808_train_신용정보.parquet 로드 완료
✅ drive/MyDrive/고급프로그래밍/train/2.신용정보/201809_train_신용정보.parquet 로드 완료
✅ drive/MyDrive/고급프로그래밍/train/2.신용정보/201812_train_신용정보.parquet 로드 완료
✅ drive/MyDrive/고급프로그래밍/train/2.신용정보/201810_train_신용정보.parquet 로드 완료
✅ drive/MyDrive/고급프로그래밍/train/2.신용정보/201811_train_신용정보.parquet 로드 완료
✅ drive/MyDrive/고급프로그래밍/train/3.승인매출정보/201807_train_승인매출정보.parquet 로드 완료
✅ drive/MyD

,폴더명,파일명,행 수,열 수,컬럼 목록,데이터 타입,결측치 비율,샘플 데이터
0,1.회원정보,drive/MyDrive/고급프로그래밍/train/1.회원정보/201809_trai...,400000,78,"[기준년월, ID, 남녀구분코드, 연령, Segment, 회원여부_이용가능, 회원여...","{'기준년월': int64, 'ID': object, '남녀구분코드': int64,...","{'기준년월': 0.0, 'ID': 0.0, '남녀구분코드': 0.0, '연령': ...","{'기준년월': {0: 201809, 1: 201809, 2: 201809, 3: ..."
1,1.회원정보,drive/MyDrive/고급프로그래밍/train/1.회원정보/201808_trai...,400000,78,"[기준년월, ID, 남녀구분코드, 연령, Segment, 회원여부_이용가능, 회원여...","{'기준년월': int64, 'ID': object, '남녀구분코드': int64,...","{'기준년월': 0.0, 'ID': 0.0, '남녀구분코드': 0.0, '연령': ...","{'기준년월': {0: 201808, 1: 201808, 2: 201808, 3: ..."
2,1.회원정보,drive/MyDrive/고급프로그래밍/train/1.회원정보/201807_trai...,400000,78,"[기준년월, ID, 남녀구분코드, 연령, Segment, 회원여부_이용가능, 회원여...","{'기준년월': int64, 'ID': object, '남녀구분코드': int64,...","{'기준년월': 0.0, 'ID': 0.0, '남녀구분코드': 0.0, '연령': ...","{'기준년월': {0: 201807, 1: 201807, 2: 201807, 3: ..."
3,1.회원정보,drive/MyDrive/고급프로그래밍/train/1.회원정보/201810_trai...,400000,78,"[기준년월, ID, 남녀구분코드, 연령, Segment, 회원여부_이용가능, 회원여...","{'기준년월': int64, 'ID': object, '남녀구분코드': int64,...","{'기준년월': 0.0, 'ID': 0.0, '남녀구분코드': 0.0, '연령': ...","{'기준년월': {0: 201810, 1: 201810, 2: 201810, 3: ..."
4,1.회원정보,drive/MyDrive/고급프로그래밍/train/1.회원정보/201812_trai...,400000,78,"[기준년월, ID, 남녀구분코드, 연령, Segment, 회원여부_이용가능, 회원여...","{'기준년월': int64, 'ID': object, '남녀구분코드': int64,...","{'기준년월': 0.0, 'ID': 0.0, '남녀구분코드': 0.0, '연령': ...","{'기준년월': {0: 201812, 1: 201812, 2: 201812, 3: ..."
5,1.회원정보,drive/MyDrive/고급프로그래밍/train/1.회원정보/201811_trai...,400000,78,"[기준년월, ID, 남녀구분코드, 연령, Segment, 회원여부_이용가능, 회원여...","{'기준년월': int64, 'ID': object, '남녀구분코드': int64,...","{'기준년월': 0.0, 'ID': 0.0, '남녀구분코드': 0.0, '연령': ...","{'기준년월': {0: 201811, 1: 201811, 2: 201811, 3: ..."
6,2.신용정보,drive/MyDrive/고급프로그래밍/train/2.신용정보/201807_trai...,400000,42,"[기준년월, ID, 최초한도금액, 카드이용한도금액, CA한도금액, 일시상환론한도금액...","{'기준년월': int64, 'ID': object, '최초한도금액': int64,...","{'기준년월': 0.0, 'ID': 0.0, '최초한도금액': 0.0, '카드이용한...","{'기준년월': {0: 201807, 1: 201807, 2: 201807, 3: ..."
7,2.신용정보,drive/MyDrive/고급프로그래밍/train/2.신용정보/201808_trai...,400000,42,"[기준년월, ID, 최초한도금액, 카드이용한도금액, CA한도금액, 일시상환론한도금액...","{'기준년월': int64, 'ID': object, '최초한도금액': int64,...","{'기준년월': 0.0, 'ID': 0.0, '최초한도금액': 0.0, '카드이용한...","{'기준년월': {0: 201808, 1: 201808, 2: 201808, 3: ..."
8,2.신용정보,drive/MyDrive/고급프로그래밍/train/2.신용정보/201809_trai...,400000,42,"[기준년월, ID, 최초한도금액, 카드이용한도금액, CA한도금액, 일시상환론한도금액...","{'기준년월': int64, 'ID': object, '최초한도금액': int64,...","{'기준년월': 0.0, 'ID': 0.0, '최초한도금액': 0.0, '카드이용한...","{'기준년월': {0: 201809, 1: 201809, 2: 201809, 3: ..."
9,2.신용정보,drive/MyDrive/고급프로그래밍/train/2.신용정보/201812_trai...,400000,42,"[기준년월, ID, 최초한도금액, 카드이용한도금액, CA한도금액, 일시상환론한도금액...","{'기준년월': int64, 'ID': object, '최초한도금액': int64,...","{'기준년월': 0.0, 'ID': 0.0, '최초한도금액': 0.0, '카드이용한...","{'기준년월': {0: 201812, 1: 201812, 2: 201812, 3: ..."


- 모든 parquet 파일의 row는 400000개, column은 카테고리 별로 동일하다.
- 카테고리 별로 column의 데이터 타입과 의미가 동일하다.
- 기준년월과 ID를 기준으로 primary key 역할이 가능하다.
- 카테고리 기준으로 join 진행

In [1]:
import pandas as pd
import glob
import os
import pyarrow.parquet as pq
import pyarrow as pa

def merge_and_save_arrow(folder_name, save_name, base_path, save_path):
    folder_path = os.path.join(base_path, folder_name)
    save_file_path = os.path.join(save_path, save_name)

    if os.path.exists(save_file_path):
        print(f"⏭️ {save_name} 이미 존재하여 건너뜁니다.")
        return

    file_list = sorted(glob.glob(os.path.join(folder_path, "*.parquet")))

    writer = None

    for i, file in enumerate(file_list):
        try:
            df = pd.read_parquet(file)

            table = pa.Table.from_pandas(df)
            if writer is None:
                writer = pq.ParquetWriter(save_file_path, table.schema)
            writer.write_table(table)

            print(f"{i+1}/{len(file_list)} - 저장 완료: {os.path.basename(file)}")

            del df  # 메모리 절약

        except Exception as e:
            print(f"⚠️ {os.path.basename(file)} 처리 중 오류 발생: {e}")

    if writer:
        writer.close()
        print(f"✅ {folder_name} 병합 완료 → 저장: {save_name}")

# 경로 설정
base_path = "/content/drive/MyDrive/고급프로그래밍/train"
save_path = "/content/drive/MyDrive/고급프로그래밍/merged"
os.makedirs(save_path, exist_ok=True)

# 병합 대상 카테고리 목록
category_map = {
    "1.회원정보": "member_all.parquet",
    "2.신용정보": "credit_all.parquet",
    "3.승인매출정보": "tx_all.parquet",
    "4.청구입금정보": "bill_all.parquet",
    "5.잔액정보": "balance_all.parquet",
    "6.채널정보": "channel_all.parquet",
    "7.마케팅정보": "mkt_all.parquet",
    "8.성과정보": "perf_all.parquet"
}

for folder, filename in category_map.items():
    merge_and_save_arrow(folder, filename, base_path, save_path)

⏭️ member_all.parquet 이미 존재하여 건너뜁니다.
⏭️ credit_all.parquet 이미 존재하여 건너뜁니다.
1/6 - 저장 완료: 201807_train_승인매출정보.parquet
2/6 - 저장 완료: 201808_train_승인매출정보.parquet
3/6 - 저장 완료: 201809_train_승인매출정보.parquet
4/6 - 저장 완료: 201810_train_승인매출정보.parquet
5/6 - 저장 완료: 201811_train_승인매출정보.parquet
6/6 - 저장 완료: 201812_train_승인매출정보.parquet
✅ 3.승인매출정보 병합 완료 → 저장: tx_all.parquet
1/6 - 저장 완료: 201807_train_청구정보.parquet
2/6 - 저장 완료: 201808_train_청구정보.parquet
3/6 - 저장 완료: 201809_train_청구정보.parquet
4/6 - 저장 완료: 201810_train_청구정보.parquet
5/6 - 저장 완료: 201811_train_청구정보.parquet
6/6 - 저장 완료: 201812_train_청구정보.parquet
✅ 4.청구입금정보 병합 완료 → 저장: bill_all.parquet
1/6 - 저장 완료: 201807_train_잔액정보.parquet
2/6 - 저장 완료: 201808_train_잔액정보.parquet
3/6 - 저장 완료: 201809_train_잔액정보.parquet
4/6 - 저장 완료: 201810_train_잔액정보.parquet
5/6 - 저장 완료: 201811_train_잔액정보.parquet
6/6 - 저장 완료: 201812_train_잔액정보.parq

## 결측치 확인

- 데이터 컬럼의 수가 너무 많아서 이상치 조정에 대한 명확한 기준을 알 수 없다.
- 결측치만이라도 조정하여 데이터 전처리를 진행

## 결측치를 처리하는 단계
1. 결측률 확인 : 결측률이 80% 이상이면 해당 column 삭제
2. column 용도에 따른 기준 정의



In [2]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import pyarrow.parquet as pq
# import os

# # 경로 설정
# merged_path = "/content/drive/MyDrive/고급프로그래밍/merged"
# file_list = sorted([f for f in os.listdir(merged_path) if f.endswith(".parquet")])

# # 결과 저장용 리스트
# summary_data = []
# missing_info = {}
# outlier_info = {}

# LIMITED_COLS = 30  # 열 제한할 파일의 최대 열 수
# LIMITED_FILE = "tx_all.parquet"  # 열 제한이 필요한 특정 파일 이름

# for file in file_list[:3]:
#     file_path = os.path.join(merged_path, file)
#     print(f"{file_path} → 데이터 요약 중...")

#     try:
#         if file == LIMITED_FILE:
#             # 열 수가 많아 다운되던 파일은 일부 열만 제한적으로 로드
#             table = pq.read_table(file_path)
#             df = table.to_pandas()
#             df = df[df.columns[:LIMITED_COLS]]  # 컬럼 제한
#         else:
#             df = pd.read_parquet(file_path)

#         # 요약
#         desc = df.describe().T
#         desc["missing_rate"] = df.isnull().mean()
#         desc["num_missing"] = df.isnull().sum()
#         desc["file"] = file
#         summary_data.append(desc)

#         # 결측치 비율
#         missing_info[file] = df.isnull().mean().sort_values(ascending=False).head(15)

#         # 이상치 비율
#         numeric_df = df.select_dtypes(include='number')
#         outlier_percent = {}
#         for col in numeric_df.columns:
#             q1 = numeric_df[col].quantile(0.25)
#             q3 = numeric_df[col].quantile(0.75)
#             iqr = q3 - q1
#             lower = q1 - 1.5 * iqr
#             upper = q3 + 1.5 * iqr
#             outlier_count = ((numeric_df[col] < lower) | (numeric_df[col] > upper)).sum()
#             outlier_percent[col] = outlier_count / len(numeric_df)
#         outlier_info[file] = pd.Series(outlier_percent).sort_values(ascending=False).head(15)

#     except Exception as e:
#         print(f"⚠️ {file} 처리 실패: {e}")

# # 시각화 - 결측치
# fig1, axes1 = plt.subplots(len(missing_info), 1, figsize=(10, len(missing_info)*4))
# for ax, (file, series) in zip(axes1, missing_info.items()):
#     sns.barplot(x=series.values, y=series.index, ax=ax)
#     ax.set_title(f"Missing Rate Top 15 - {file}")
#     ax.set_xlabel("Missing Rate")
# plt.tight_layout()

# # 시각화 - 이상치
# fig2, axes2 = plt.subplots(len(outlier_info), 1, figsize=(10, len(outlier_info)*4))
# for ax, (file, series) in zip(axes2, outlier_info.items()):
#     sns.barplot(x=series.values, y=series.index, ax=ax)
#     ax.set_title(f"Outlier Rate Top 15 - {file}")
#     ax.set_xlabel("Outlier Rate")
# plt.tight_layout()
# plt.show()

# # # 요약 데이터프레임 출력
# # summary_all = pd.concat(summary_data)
# # import ace_tools as tools; tools.display_dataframe_to_user(name="병합된 파일 요약 통계", dataframe=summary_all)


- treshhold = 70%
- IQR 방식으로 이상치 제거

In [ ]:
def remove_outliers_iqr(df, threshold=1.5):
    df_clean = df.copy()
    for col in df_clean.select_dtypes(include='number').columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1

        # 이상치 범위 설정
        lower = Q1 - threshold * IQR
        upper = Q3 + threshold * IQR

        # 이상치 제거
        df_clean = df_clean[(df_clean[col] >= lower) & (df_clean[col] <= upper)]

    return df_clean


# 정제된 결과를 다시 dfs에 반영
for name, df in dfs.items():
    print("="*100)
    print(f"📁 [{name}] 이상치 및 결측치 제거")
    print("="*100)

    before_shape = df.shape

    # 1. 결측치 70% 이상 있는 행 제거
    thresh = int(df.shape[1] * 0.7)
    df_clean = df.dropna(thresh=thresh)

    # 2. IQR 기반 이상치 제거
    df_clean = remove_outliers_iqr(df_clean)

    after_shape = df_clean.shape
    dfs[name] = df_clean  # 덮어쓰기

    print(f"✅ 정제 완료: {before_shape} → {after_shape}\n")

📁 [회원정보] 이상치 및 결측치 제거
✅ 정제 완료: (400000, 78) → (30849, 78)

📁 [신용정보] 이상치 및 결측치 제거
✅ 정제 완료: (400000, 42) → (9781, 42)

📁 [승인매출정보] 이상치 및 결측치 제거
✅ 정제 완료: (400000, 406) → (0, 406)

📁 [청구입금정보] 이상치 및 결측치 제거
✅ 정제 완료: (400000, 46) → (47417, 46)

📁 [잔액정보] 이상치 및 결측치 제거
✅ 정제 완료: (400000, 82) → (48, 82)

📁 [채널정보] 이상치 및 결측치 제거


KeyboardInterrupt: 